In [2]:
"""
from datetime import date, timedelta

start_date = date.today() - timedelta(30)

print(start_date)
"""

StatementMeta(, 42c937b0-9157-4c37-8e9b-ef155f3c5863, 6, Finished, Available, Finished)

2024-08-10


In [3]:
from pyspark.sql.functions import when,col,udf
from pyspark.sql.functions import StringType
import reverse_geocoder as rg

df = spark.read.table("earthquake_events_silver").filter(col('time') > start_date)


StatementMeta(, 42c937b0-9157-4c37-8e9b-ef155f3c5863, 7, Finished, Available, Finished)

In [4]:
def getCountryCode(lat, lon):
    # Convert latitude and longitude to float and search for the country code using reverse geocoding
    coordinates = (float(lat), float(lon))
    
    # Return the country code ('cc') from the reverse geocoding result
    return rg.search(coordinates)[0].get('cc')

# Register the getCountryCode function as a Spark UDF (User Defined Function) that returns a StringType
getCountryCode_udf = udf(getCountryCode, StringType())


StatementMeta(, 42c937b0-9157-4c37-8e9b-ef155f3c5863, 8, Finished, Available, Finished)

In [5]:
#adding country_code and city attributes
df_with_location = df.withColumn('country_code',getCountryCode_udf (col('latitude'),col('longitude')))

StatementMeta(, 42c937b0-9157-4c37-8e9b-ef155f3c5863, 9, Finished, Available, Finished)

In [20]:
#adding significance classification
df_with_sig = df_with_location.withColumn('significance_class',
                                           when(col('significance') < 100, "Low").
                                           when((col('significance') >= 100) & (col('significance') < 500), "Moderate" ).
                                           otherwise('High')


)


StatementMeta(, 42c937b0-9157-4c37-8e9b-ef155f3c5863, 24, Finished, Available, Finished)

In [30]:
#appending the data to gold table
df_with_sig.write.mode('append').saveAsTable('earthquake_events_gold')

StatementMeta(, 42c937b0-9157-4c37-8e9b-ef155f3c5863, 34, Finished, Available, Finished)